In [99]:
import numpy as np
import uproot
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier, BaggingClassifier
from sklearn.multioutput import MultiOutputRegressor
import sklearn
from random import random
from sklearn.model_selection import cross_validate
import matplotlib
from waveform_methods import binning
import pandas as pd

from scipy.stats import norm
from sklearn.base import clone
import pickle
from statistics import mode
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold

In [100]:
file = uproot.open('/home/andy/12360.root')
file2 = uproot.open('/home/andy/12362.root')

In [101]:
print(file['Stoch_Reco'].keys())

[b'Run', b'Event', b'SubEvent', b'SubEventStream', b'SubEventStream_String', b'exists', b'eloss_1000', b'eloss_1500', b'eloss_1600', b'eloss_1700', b'eloss_1800', b'eloss_1900', b'eloss_2000', b'eloss_2100', b'eloss_2200', b'eloss_2300', b'eloss_2400', b'eloss_3000', b'a_estim', b'a_estim_err', b'e0_estim', b'e0_estim_err', b'stoch_energy', b'rel_stoch_energy', b'total_stoch_energy', b'total_rel_stoch_energy', b'highest_stoch_energy', b'highest_rel_stoch_energy', b'chi2', b'chi2_red', b'stoch_depth', b'n_he_stoch', b'fit_status', b'fit_status_String']


In [102]:
Eloss_keys = file['Stoch_Reco'].keys()[6:18]
for i in range(len(Eloss_keys)):
    if i == 0:
        Eloss1 = file['Stoch_Reco'][Eloss_keys[i]].array()
    else:
        Eloss1 += file['Stoch_Reco'][Eloss_keys[i]].array()
Eloss1 = Eloss1/len(Eloss_keys)

Eloss_keys = file2['Stoch_Reco'].keys()[6:18]
for i in range(len(Eloss_keys)):
    if i == 0:
        Eloss2 = file2['Stoch_Reco'][Eloss_keys[i]].array()
    else:
        Eloss2 += file2['Stoch_Reco'][Eloss_keys[i]].array()
Eloss2 = Eloss2/len(Eloss_keys)

Eloss = np.append(Eloss1,Eloss2)

In [103]:
cut = []
for i in file['IT73AnalysisInIceQualityCuts'].keys()[6:11]:
    cut.append(file['IT73AnalysisInIceQualityCuts'][i].array())
cut2 = list(zip(cut))
for i in range(len(cut2)):
    if i == 0:
        final_cut = cut2[i][0]
    else:
        final_cut = [j and k for j,k in zip(final_cut,cut2[i][0])]
        
cut = []
for i in file2['IT73AnalysisInIceQualityCuts'].keys()[6:11]:
    cut.append(file2['IT73AnalysisInIceQualityCuts'][i].array())
cut2 = list(zip(cut))
for i in range(len(cut2)):
    if i == 0:
        final_cut2 = cut2[i][0]
    else:
        final_cut2 = [j and k for j,k in zip(final_cut,cut2[i][0])]
        
in_ice_cuts = np.append(final_cut,final_cut2)

In [104]:
Xmax1 = file['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file['MCPrimaryInfo']['longNumCharged'].array()
Depth1 = file['MCPrimaryInfo']['longDepth'].array()
chi2_1 = file['CurvatureOnlyParams']['chi2_time'].array()
red1 = file['MCPrimaryInfo']['ghRedChiSqr'].array()
mass1 = [1 for i in range(len(red1))]
        
Xmax2 = file2['MCPrimaryInfo']['ghMaxDepth'].array()
valueDepth = file2['MCPrimaryInfo']['longNumCharged'].array()
Depth2 = file2['MCPrimaryInfo']['longDepth'].array()
chi2_2 = file2['CurvatureOnlyParams']['chi2_time'].array()
red2 = file2['MCPrimaryInfo']['ghRedChiSqr'].array()
mass2 = [4 for i in range(len(red2))]


In [105]:
S125_1 = file['LaputopParams']['s125'].array()
S125_2 = file2['LaputopParams']['s125'].array()
A1 = file['CurvatureOnlyParams']['A'].array()
A2 = file2['CurvatureOnlyParams']['A'].array()
D1 = file['CurvatureOnlyParams']['D'].array()
D2 = file2['CurvatureOnlyParams']['D'].array()
N1 = file['CurvatureOnlyParams']['N'].array()
N2 = file2['CurvatureOnlyParams']['N'].array()
beta1 = file['LaputopParams']['beta'].array()
beta2 = file2['LaputopParams']['beta'].array()
zenith1 = file['Laputop']['zenith'].array()
zenith2 = file2['Laputop']['zenith'].array()
energy1 = file['MCPrimary']['energy'].array()
energy2 = file2['MCPrimary']['energy'].array()
stoch1_1 = file['Stoch_Reco']['n_he_stoch'].array()
stoch1_2 = file2['Stoch_Reco']['n_he_stoch'].array()
stoch2_1 = file['Stoch_Reco2']['n_he_stoch'].array()
stoch2_2 = file2['Stoch_Reco2']['n_he_stoch'].array()

slc_charge1 = file['LaputopSLCPE']['charge'].array()
slc_charge2 = file2['LaputopSLCPE']['charge'].array()

slc_charge1 = [np.sum(i[np.isnan(i)==False]) for i in slc_charge1]
slc_charge2 = [np.sum(i[np.isnan(i)==False]) for i in slc_charge2]

hlc_charge1 = file['LaputopHLCPE']['charge'].array()
hlc_charge2 = file2['LaputopHLCPE']['charge'].array()

hlc_charge1 = [np.sum(i[np.isnan(i) ==False]) for i in hlc_charge1]
hlc_charge2 = [np.sum(i[np.isnan(i)==False]) for i in hlc_charge2]

Qtot1 = [i+j for i,j in zip(slc_charge1,hlc_charge1)]
Qtot2 = [i+j for i,j in zip(slc_charge2,hlc_charge2)]

In [106]:
A = np.append(A1,A2)
D = np.append(D1,D2)
N = np.append(N1,N2)
S125 = np.log10(np.append(S125_1,S125_2))
beta = np.log10(np.append(beta1,beta2))
chi2 = np.append(chi2_1,chi2_2)
red = np.append(red1,red2)
zenith = np.cos(np.append(zenith1,zenith2))
Xmax = np.append(Xmax1,Xmax2)
red = np.append(red1,red2)
mass = np.append(mass1,mass2)
slc_charge = np.append(slc_charge1,slc_charge2)
hlc_charge = np.append(hlc_charge1,hlc_charge2)
qtot = np.append(Qtot1,Qtot2)
stoch1 = np.append(stoch1_1,stoch1_2)
stoch2 = np.append(stoch2_1,stoch2_2)
for i in range(len(stoch1)):
    if not in_ice_cuts[i]:
        stoch1[i] = 0
        stoch2[i] = 0

energy_loss = Eloss
for i in range(len(energy_loss)):
    if ((energy_loss[i] != 0 and in_ice_cuts[i]) and np.isfinite(energy_loss[i])) and not np.isnan(energy_loss[i]):
        energy_loss[i] = np.log10(abs(energy_loss[i]))
    else:
        energy_loss[i] = 0
        
energy = np.log10(np.append(energy1,energy2))

In [107]:
values = [mass,
            energy_loss,
            stoch1,
            stoch2,
            A,
            D,
            N,
            S125,
            beta,
            zenith,
            chi2,
            slc_charge,
            hlc_charge,
            qtot
            ]

print(np.corrcoef(values))

[[ 1.00000000e+00 -1.67488960e-01 -1.59570024e-01 -1.19173749e-01
  -1.91653200e-01 -1.32195338e-01  1.25447376e-01  4.29808393e-02
  -2.00833332e-01  1.33948905e-02  5.60756736e-04  2.68353149e-01
  -1.26407490e-02 -1.08003071e-02]
 [-1.67488960e-01  1.00000000e+00  8.08854556e-01  3.82294608e-01
   6.20143587e-02 -5.45574992e-02 -4.71564816e-02  1.91778746e-01
   1.59400512e-01  3.30219081e-01 -2.20658129e-02 -6.91919950e-02
   1.49829065e-01  1.49196148e-01]
 [-1.59570024e-01  8.08854556e-01  1.00000000e+00  5.23775355e-01
   8.03260903e-02 -1.17366307e-02 -4.60883112e-02  1.29606149e-01
   1.45734553e-01  3.12321477e-01 -1.32530912e-02 -8.71389675e-02
   1.18129195e-01  1.17408360e-01]
 [-1.19173749e-01  3.82294608e-01  5.23775355e-01  1.00000000e+00
   9.62204274e-02  5.95681240e-02 -4.57410568e-02 -1.92431528e-02
   9.24728093e-02  2.00266772e-01  3.91724063e-03 -1.02598803e-01
   1.45791996e-03  7.57915232e-04]
 [-1.91653200e-01  6.20143587e-02  8.03260903e-02  9.62204274e-02
  

In [146]:
import random
#model coinc:slc_charge,A,N,beta,energy_loss,stoch1,zenith
#model not coinc:slc_charge,beta,A,zenith
input_variable2_old = np.array(list(zip(slc_charge,A)))[in_ice_cuts]
output_new_old = mass[in_ice_cuts]
energy_value = energy[in_ice_cuts]

validation = []
output = []

for i in range(len(input_variable2_old)):
    if random.random() < 0.1:
        validation.append(i)
    else:
        output.append(i)
        
validation_x = input_variable2_old[validation]
validation_y = output_new_old[validation]
input_variable = np.array(input_variable2_old)[output]
output_new = np.array(output_new_old)[output]
energy_value = energy_value[output]

In [147]:
weight = (qtot**0.5)[in_ice_cuts][output]

In [148]:
index = list(range(len(weight)))
test_index = []
train_index = []
for i in index:
    if random.random() < 0.2:
        test_index.append(i)
    else: 
        train_index.append(i)
test_index = np.array(test_index)
train_index = np.array(train_index)

In [154]:
from sklearn.metrics import log_loss
import pickle
#23 without any filters
for j in range(1,30):
    rng = np.random.RandomState(1)
    proton = []
    iron = []
    regr_2 = None
    regr_2 = AdaBoostClassifier(DecisionTreeClassifier(max_depth = 10,splitter='random',class_weight='balanced',max_leaf_nodes = 5, min_samples_leaf = j),
                                            n_estimators=2,random_state=rng)

    regr_2.fit(input_variable[train_index],output_new[train_index],sample_weight=weight[train_index])
    predicted_value = regr_2.predict(input_variable[test_index])
    array_split = np.array_split(sorted(list(zip(predicted_value,energy_value[test_index],output_new[test_index])),key=lambda x:x[1]),20)
    for value in array_split:
        cm = confusion_matrix(list(zip(*value))[2],list(zip(*value))[0])
        proton.append(cm[0][0]/np.sum(cm[0]))
        iron.append(cm[1][1]/np.sum(cm[1]))
    print(np.mean(proton),max(proton)-min(proton),np.mean(iron),max(iron)-min(iron),i,j)
    #pickle.dump(regr_2,open('model_coinc.sav','wb'))

0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 1
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 2
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 3
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 4
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 5
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 6
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 7
0.9224579742990761 0.2731707317073171 0.21284555187796522 0.5714285714285714 23441 8
0.6537800912692153 0.8264808362369338 0.6155432321189702 0.8571428571428572 23441 9
0.6537800912692153 0.8264808362369338 0.6155432321189702 0.8571428571428572 23441 10
0.6537800912692153 0.8264808362369338 0.6155432321189702 0.8571428571428572 23441 11
0.6537800912692153 0.8264808362369338 0.6155432321189702 0.8571428